In [ ]:
# selenium 불러오기
from selenium import webdriver
import time

# chrome창(웹드라이버) 열기
driver = webdriver.Chrome("./chromedriver")

# 실행할 웹페이지 불러오기(구글 지도 예시)
driver.get("https://www.google.com/maps/")

# 선택자로 페이지 요소 선택하기(단일)
driver.find_element_by_css_selector("선택자")

# 선택자로 페이지 요소 선택하기(다중)
driver.find_elements_by_css_selector("선택자")

# 해당 요소에 문자 입력하기(ex. 버튼)
button.send_keys("텍스트")

# 해당 요소 클릭하기
button.click()

# 의도적으로 3초 쉬어주기
time.sleep(3)

# 여러 페이지(999)에서 반복하기
for i in range(999):
    # 시간 지연
    time.sleep(3)

    # 컨테이너(가게) 데이터 수집 // div.section-result-content
    stores = driver.find_elements_by_css_selector("div.section-result-content")

    for s in stores:
        # 가게 이름 데이터 수집 // h3.section-result-title
        title = s.find_element_by_css_selector("h3.section-result-title").text

        # 평점 데이터 수집 // span.cards-rating-score
        # 평점이 없는 경우 에러 처리
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"

        # 가게 주소 데이터 수집 // span.section-result-location
        addr = s.find_element_by_css_selector("span.section-result-location").text

        print(title, "/", score, "/", addr)

    # 다음페이지 버튼 클릭 하기
    # 다음페이지가 없는 경우(데이터 수집 완료) 에러 처리
    try:
        nextpage = driver.find_element_by_css_selector("button#n7lv7yjyC35__section-pagination-button-next")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time

query = '미용실' #검색어
sortingOrder = 'reviewCount' #정렬방식
arrLat = [] #위도 y
arrLng = [] #경도 x
lat = '35.22811824007319' #창원
lng = '128.6818464845621'

link = 'https://m.place.naver.com/hairshop/list?sm=mtp_hty.top&where=m&query='
link += query + '&x=' + lng + '&y=' + lat + '&level=top&entry=pll&sortingOrder=' + sortingOrder
link += '&gender=f&listViewQuery=' + query
        
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("./chromedriver")

# 네이버 플레이스 링크 접속
driver.get(link)
time.sleep(3)

last_height = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2)").size
hover = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(1) > div._15UxR > div > div > a._2QhH-._1-QNb")
hover.send_keys(Keys.ENTER)

while True:
    for i in range(30):
        hover.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    size = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2)").size
    if size['height'] == last_height['height']:
        time.sleep(30)
        if size['height'] == last_height['height']:
            print("데이터 로드 완료")
            break
        print('네트워크 쫌 느림')
    last_height = size
    print(size['height'])


# for i in range(999):
#     # 컨테이너(가게) 데이터 수집 // div.section-result-content
#     stores = driver.find_elements_by_css_selector("div.section-result-content")

# 크롬창 닫기
#driver.close()

21019
29426
41816
53883
59820
71865
83494
94235
103409
108084
109467
네트워크 쫌 느림
데이터 로드 완료
